<a href="https://colab.research.google.com/github/MiM0ulay/RiskMetrics/blob/main/ETHUSDRiskMetricFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nasdaq-data-link


In [ ]:
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import nasdaqdatalink
import yfinance as yf

In [ ]:

# Download data
df = yf.download(tickers='ETH-USD', start='2016-01-01', interval='1d')

# Debugging: Check the column names
print("Column names before processing:", df.columns)

# Reset the index and flatten column names (if needed)
df.reset_index(inplace=True)

# Handle multi-level columns (flatten them if they exist)
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

# Debugging: Check flattened column names
print("Column names after flattening:", df.columns)

# Rename columns for consistency
if 'Open' in df.columns:
    df.rename(columns={'Date': 'date', 'Open': 'value'}, inplace=True)
elif 'value' not in df.columns:
    raise KeyError("The required columns ('date', 'value') are not found in the DataFrame.")

# Keep necessary columns
df = df[['date', 'value']].copy()

# Sort by date
df.sort_values(by='date', inplace=True)

# Calculate moving average
moving_average_days = 365
df['MA'] = df['value'].rolling(moving_average_days, min_periods=1).mean()

# Ensure no NaN values
df = df.dropna().reset_index(drop=True)

# Calculate Preavg
diminishing_factor = 0.395
df['Preavg'] = (np.log(df['value']) - np.log(df['MA'])) * (np.arange(len(df)) + 1) ** diminishing_factor

# Print results
print(df.head())


[*********************100%***********************]  1 of 1 completed

Column names before processing: MultiIndex([('Adj Close', 'ETH-USD'),
            (    'Close', 'ETH-USD'),
            (     'High', 'ETH-USD'),
            (      'Low', 'ETH-USD'),
            (     'Open', 'ETH-USD'),
            (   'Volume', 'ETH-USD')],
           names=['Price', 'Ticker'])
Column names after flattening: Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
        date       value          MA    Preavg
0 2017-11-09  308.644989  308.644989  0.000000
1 2017-11-10  320.670990  314.657990  0.024891
2 2017-11-11  298.585999  309.300659 -0.054412
3 2017-11-12  314.690002  310.647995  0.022353
4 2017-11-13  307.024994  309.923395 -0.017743


In [ ]:
print("Column names:", df.columns)


Column names: Index(['date', 'value', 'MA', 'Preavg'], dtype='object')


In [ ]:
# Normalization to 0-1 range
df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())

In [ ]:
price_per_risk = {
    round(risk, 1):round(np.exp(
        (risk * (df['Preavg'].cummax().iloc[-1] - (cummin := df['Preavg'].cummin().iloc[-1])) + cummin) / df.index[-1]**diminishing_factor + np.log(df['MA'].iloc[-1])
    ))
    for risk in np.arange(0.0, 1.0, 0.1)
}

In [ ]:
# # Exclude the first 1000 days from the dataframe, because it's pure chaos
AnnotationText = f"Updated: {df.index[-1]} | Price: {round(df['value'].iloc[-1])} | Risk: {round(df['avg'].iloc[-1], 2)}"

In [ ]:
# Plot BTC-USD and Risk on a logarithmic chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Add BTC-USD and Risk data to the figure
fig.add_trace(go.Scatter(x=df['date'], y=df['value'], name='Price', line=dict(color='gold')))
fig.add_trace(go.Scatter(x=df['date'], y=df['avg'],   name='Risk',  line=dict(color='white')), secondary_y=True)

# Add green (`accumulation` or `buy`) rectangles to the figure
opacity = 0.2
for i in range(5, 0, -1):
    opacity += 0.05
    fig.add_hrect(y0=i*0.1, y1=((i-1)*0.1), line_width=0, fillcolor='green', opacity=opacity, secondary_y=True)

# Add red (`distribution` or `sell`) rectangles to the figure
opacity = 0.2
for i in range(6, 10):
    opacity += 0.1
    fig.add_hrect(y0=i*0.1, y1=((i+1)*0.1), line_width=0, fillcolor='red', opacity=opacity, secondary_y=True)

fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_yaxes(title='Risk', type='linear', secondary_y=True, showgrid=True, tick0=0.0, dtick=0.1, range=[0, 1])
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot BTC-USD colored according to Risk values on a logarithmic chart
fig = px.scatter(df, x='date', y='value', color='avg', color_continuous_scale='jet')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot Predicting BTC price according to specific risk
fig = go.Figure(data=[go.Table(
    header=dict(values=['Risk', 'Price'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[list(price_per_risk.keys()), list(price_per_risk.values())],
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])
fig.update_layout(width=500, height=500, title={'text': 'Price according to specific risk', 'y': 0.9, 'x': 0.5})
fig.show()

In [ ]:
df.tail(60)

,date,value,MA,Preavg,avg
2546,2024-10-29,2565.417969,2831.909402,-2.189118,0.405849
2547,2024-10-30,2637.640137,2834.176549,-1.592125,0.418625
2548,2024-10-31,2657.213135,2836.481593,-1.446571,0.421741
2549,2024-11-01,2515.870361,2838.313402,-2.672385,0.395506
2550,2024-11-02,2512.208252,2840.262147,-2.720297,0.394480
2551,2024-11-03,2491.094482,2842.064135,-2.921871,0.390166
2552,2024-11-04,2456.095215,2843.704408,-3.248821,0.383169
2553,2024-11-05,2397.036377,2845.082515,-3.799785,0.371377
2554,2024-11-06,2422.539307,2846.512487,-3.576805,0.376149
2555,2024-11-07,2724.005859,2848.805327,-0.993637,0.431434
